# LeNet

接下来的几天我们将主要精力用来重温以下经典。
![](./img/history.png)

卷积神经网络的鼻祖，Yan Lecun在1998年提出来的，卷积神经网络，比较简单，我们来实现一下

![](./img/lenet.jpg)

In [3]:
import torch
from torch.autograd import Variable
import torch.nn as nn
from torchvision import transforms, datasets

# load mnist
def minst_loader():
    # load mnist
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])  

    train_set = datasets.MNIST('./data', train = True, download = True, transform = transform)
    test_set = datasets.MNIST('./data', train = False, download = True, transform = transform)

    train_loader = torch.utils.data.DataLoader(train_set, batch_size = 32, shuffle = True)
    test_loader = torch.utils.data.DataLoader(train_set, shuffle = False)
    return (train_loader, test_loader)


In [4]:
help(nn.Conv2d)

Help on class Conv2d in module torch.nn.modules.conv:

class Conv2d(_ConvNd)
 |  Applies a 2D convolution over an input signal composed of several input
 |  planes.
 |  
 |  In the simplest case, the output value of the layer with input size
 |  :math:`(N, C_{in}, H, W)` and output :math:`(N, C_{out}, H_{out}, W_{out})`
 |  can be precisely described as:
 |  
 |  .. math::
 |  
 |      \begin{array}{ll}
 |      out(N_i, C_{out_j})  = bias(C_{out_j})
 |                     + \sum_{{k}=0}^{C_{in}-1} weight(C_{out_j}, k)  \star input(N_i, k)
 |      \end{array}
 |  
 |  where :math:`\star` is the valid 2D `cross-correlation`_ operator
 |  
 |  | :attr:`stride` controls the stride for the cross-correlation.
 |  | If :attr:`padding` is non-zero, then the input is implicitly zero-padded
 |    on both sides for :attr:`padding` number of points.
 |  | :attr:`dilation` controls the spacing between the kernel points; also
 |    known as the à trous algorithm. It is harder to describe, but this `

In [ ]:
# 尝试使用Sequential
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 6, 2), # c1
            nn.MaxPool2d()
        )